In [ ]:
!pip install psycopg2 

In [ ]:
import psycopg2
import pandas as pd

In [ ]:
def create_database():
    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1  port=5433 dbname=postgres user=postgres, password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

# create sparikify database with UTP-8 encoding
    cur.execute("DROP DATABASE accounts")
    cur.execute("CREATE DATABASE accounts")
    
# close connection to default database 
    conn.close()

# connect to sparkify database 
    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=root")
    cur = conn.cursor()  
       
    return cur, conn

In [ ]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [ ]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [ ]:
AccountsCountry = pd.read_csv("/kaggle/input/wealth/Wealth-AccountsCountry.csv")

In [ ]:
AccountsCountry.head()

In [ ]:
AccountsCountry_clean = AccountsCountry[['Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit']]

In [ ]:
AccountsCountry_clean.head()

In [ ]:
AccountsCountry_clean_copy = AccountsCountry_clean.copy()

new_column_name = 'Country Code'
AccountsCountry_clean_copy.rename(columns={'Code': new_column_name}, inplace=True)

In [ ]:
AccountsCountry_clean.head()

In [ ]:
AccountsData = pd.read_csv("/kaggle/input/wealth/Wealth-AccountData.csv")

In [ ]:
AccountsData.head()

In [ ]:
AccountsData.columns

In [ ]:
AccountsData = AccountsData.drop(['1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]','2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]','2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]','2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'], axis=1)

In [ ]:
# Create a dictionary to map old column names to new column names
column_name_mapping = {
    'Series Name': 'Indicator Name',
    'Series Code': 'Indicator Code',
    '1995 [YR1995]': '1995',
    '2000 [YR2000]': '2000',
    '2005 [YR2005]': '2005',
    '2010 [YR2010]': '2010',
    '2014 [YR2014]': '2014',
}

AccountsData.rename(columns=column_name_mapping, inplace=True)

In [ ]:
AccountsData.columns

In [ ]:
AccountsSeries = pd.read_csv("/kaggle/input/wealth/Wealth-AccountSeries.csv")

In [ ]:
AccountsSeries.columns

In [ ]:
# Create a copy of the DataFrame to avoid the SettingWithCopyWarning
AccountsSeries_copy = AccountsSeries.copy()

# Specify the new column name
new_column_name = 'Series Code'

# Use the 'rename()' method on the copied DataFrame to rename the column
AccountsSeries_copy.rename(columns={'Code': new_column_name}, inplace=True)

In [ ]:
AccountsSeries_copy= AccountsSeries_copy[['Series Code', 'Topic', 'Indicator Name']]

In [ ]:
import numpy as np

In [ ]:
AccountsSeries_copy['Short definition'] = np.nan  

In [ ]:
AccountsSeries_copy.head()

In [ ]:
cur, conn = create_database()

In [ ]:
songplay_table_create = ("""CREATE TABLE IF NOT EXITS accountscountry(
country_code VARCHAR PRIMARY KEY,
short_name VARCHAR,
table_name VARCHAR
long_name VARCHAR
currency_unit VARCHAR)""")

In [ ]:
cur.execute(songplay_table_create)
conn.commit()

In [ ]:
accounts_data_table_create = ("""CREATE TABLE IF NOT EXITS accountsdata(
country_name VARCHAR, 
country_code VARCHAR,
indicator_name VARCHAR,
indicator_code VARCHAR
year_1995 numeric,
year_2000 numeric,
year_2005 numeric,
year_2010 numeric,
year_2014 numeric
)""")
cur.execute(accounts_data_table_create)
conn.commit()

In [ ]:
accountseries_data_table_create = ("""CREATE TABLE IF NOT EXITS accountsdata(
series_code VARCHAR, 
topic VARCHAR,
indicator_name VARCHAR,
short_definition VARCHAR
)""")
cur.execute(accountseries_data_table_create)
conn.commit()

In [ ]:
songplay_table_create = ("""INSERT INTO accountscountry(
country_code,
short_name VARCHAR,
table_name VARCHAR
long_name VARCHAR
currency_unit)
VALUES (%s, %s, %s, %s, %s)""")

In [ ]:
for i, row in AccountsCountry_clean.iterrow():
    cur.execute(accounts_data_table_insert, list(row))

In [ ]:
conn.commit()

In [ ]:
accounts_data_table_create = ("""INSERT INTO accountsdata(
country_name, 
country_code,
indicator_name,
indicator_code
year_1995,
year_2000,
year_2005,
year_2010,
year_2014)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""")

In [ ]:
for i, row in AccountsData.iterrow():
    cur.execute(accounts_data_table_insert, list(row))

In [ ]:
accountseries_data_table_create = ("""INSERT INTO accountsdata(
series_code, 
topic,
indicator_name,
short_definition)
VALUES (%s, %s, %s, %s)""")

In [ ]:
for i, row in AccountsSeries.iterrow():
    cur.execute(accountseries_data_table_insert, list(row))

In [ ]:
conn.commit()